<a href="https://colab.research.google.com/github/Emerson-Foco/Chat_Service_Evaluation_AI_Gemini_API/blob/main/Chat_Service_Evaluation_AI_Gemini_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Steps

- [ ] Install and import Google SDK

In [4]:
!pip install -q -U google-generativeai

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

- [ ] Import Goodle SDK

In [5]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

- [ ] Setting your API key by *userdata.get*

In [6]:
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')

genai.configure(api_key=api_key)

# Setting your employeer

- [ ] Employeer info

In [ ]:
employeer_name = 'Raquel'
job_position = 'atendente'
company_name = 'Google'
products_to_offer = 'AI Gemini'

employeer_info = {f'Você é {job_position} de chat e trabalha para {company_name} '
                  f'com os produtos {products_to_offer}, realize um atendimento natural, humanizado'
                  'sem assustar o cliente. Você não pode mentir e nem inventar.'
                  ' O que você não souber, diga que não possui a informação'}

- [ ] Employeer model settings

In [ ]:
employeer_generation_config = {
    'candidate_count': 1,
    'temperature': 0.9,
}

employeer_safety_settings = {
    'HARASSMENT': 'BLOCK_NONE',
    'HATE': 'BLOCK_NONE',
    'SEXUAL': 'BLOCK_NONE',
    'DANGEROUS': 'BLOCK_NONE',
}

employeer_model = genai.GenerativeModel('gemini-pro', generation_config=employeer_generation_config, safety_settings=employeer_safety_settings)